# Starting Out and Loading Data

We're going to get started by loading up yt.  This next command brings all of the libraries into memory and sets up our environment.

In [6]:
import yt

Now that we've loaded yt, we can load up some data.  Let's load the `IsolatedGalaxy` dataset.

In [31]:
import analysis_schema

In [7]:
ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")

yt : [INFO     ] 2020-05-29 14:19:49,554 Parameters: current_time              = 0.0060000200028298
yt : [INFO     ] 2020-05-29 14:19:49,554 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2020-05-29 14:19:49,556 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2020-05-29 14:19:49,556 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2020-05-29 14:19:49,561 Parameters: cosmological_simulation   = 0.0


## Fields and Facts

When you call the `load` function, yt tries to do very little -- this is designed to be a fast operation, just setting up some information about the simulation.  Now, the first time you access the "index" it will read and load the mesh and then determine where data is placed in the physical domain and on disk.  Once it knows that, yt can tell you some statistics about the simulation:

In [8]:
ds.print_stats()

Parsing Hierarchy : 100%|██████████| 173/173 [00:00<00:00, 4356.37it/s]
yt : [INFO     ] 2020-05-29 14:19:52,489 Gathering a field list (this may take a moment.)


level	# grids	       # cells	     # cells^3
----------------------------------------------
  0	     1	         32768	            32
  1	     8	         34304	            33
  2	     8	        181888	            57
  3	     8	        646968	            87
  4	    15	        947856	            99
  5	    51	        874128	            96
  6	    18	        786328	            93
  7	    28	        446776	            77
  8	    36	        209400	            60
----------------------------------------------
   	   173	       4160416


t = 6.00002000e-03 = 1.39768066e+16 s = 4.42898275e+08 years

Smallest Cell:
	Width: 1.221e-04 Mpc
	Width: 1.221e+02 pc
	Width: 2.518e+07 AU
	Width: 3.767e+20 cm


yt can also tell you the fields it found on disk:

In [9]:
ds.field_list

[('all', 'creation_time'),
 ('all', 'dynamical_time'),
 ('all', 'metallicity_fraction'),
 ('all', 'particle_index'),
 ('all', 'particle_mass'),
 ('all', 'particle_position_x'),
 ('all', 'particle_position_y'),
 ('all', 'particle_position_z'),
 ('all', 'particle_type'),
 ('all', 'particle_velocity_x'),
 ('all', 'particle_velocity_y'),
 ('all', 'particle_velocity_z'),
 ('enzo', 'Average_creation_time'),
 ('enzo', 'Bx'),
 ('enzo', 'By'),
 ('enzo', 'Bz'),
 ('enzo', 'Cooling_Time'),
 ('enzo', 'Dark_Matter_Density'),
 ('enzo', 'Density'),
 ('enzo', 'Electron_Density'),
 ('enzo', 'Forming_Stellar_Mass_Density'),
 ('enzo', 'Galaxy1Colour'),
 ('enzo', 'Galaxy2Colour'),
 ('enzo', 'HII_Density'),
 ('enzo', 'HI_Density'),
 ('enzo', 'HeIII_Density'),
 ('enzo', 'HeII_Density'),
 ('enzo', 'HeI_Density'),
 ('enzo', 'MBHColour'),
 ('enzo', 'Metal_Density'),
 ('enzo', 'PhiField'),
 ('enzo', 'Phi_pField'),
 ('enzo', 'SFR_Density'),
 ('enzo', 'Star_Particle_Density'),
 ('enzo', 'Temperature'),
 ('enzo', '

And, all of the fields it thinks it knows how to generate:

In [10]:
ds.derived_field_list

[('all', 'age'),
 ('all', 'creation_time'),
 ('all', 'dynamical_time'),
 ('all', 'mesh_id'),
 ('all', 'metallicity_fraction'),
 ('all', 'particle_angular_momentum'),
 ('all', 'particle_angular_momentum_magnitude'),
 ('all', 'particle_angular_momentum_x'),
 ('all', 'particle_angular_momentum_y'),
 ('all', 'particle_angular_momentum_z'),
 ('all', 'particle_cylindrical_velocity_theta'),
 ('all', 'particle_cylindrical_velocity_z'),
 ('all', 'particle_index'),
 ('all', 'particle_mass'),
 ('all', 'particle_ones'),
 ('all', 'particle_position'),
 ('all', 'particle_position_cylindrical_radius'),
 ('all', 'particle_position_cylindrical_theta'),
 ('all', 'particle_position_cylindrical_z'),
 ('all', 'particle_position_relative'),
 ('all', 'particle_position_relative_x'),
 ('all', 'particle_position_relative_y'),
 ('all', 'particle_position_relative_z'),
 ('all', 'particle_position_spherical_phi'),
 ('all', 'particle_position_spherical_radius'),
 ('all', 'particle_position_spherical_theta'),
 ('al

yt can also transparently generate fields.  However, we encourage you to examine exactly what yt is doing when it generates those fields.  To see, you can ask for the source of a given field.

In [11]:
print(ds.field_info["gas", "vorticity_x"].get_source())

    def _vorticity_x(field, data):
        vz = data[ftype, "relative_velocity_z"]
        vy = data[ftype, "relative_velocity_y"]
        f  = ((vz[sl_center,sl_right,sl_center] -
               vz[sl_center,sl_left,sl_center]) /
              (div_fac*just_one(data["index", "dy"])))
        f -= ((vy[sl_center,sl_center,sl_right] -
               vy[sl_center,sl_center,sl_left]) /
              (div_fac*just_one(data["index", "dz"])))
        new_field = data.ds.arr(
            np.zeros_like(vz, dtype=np.float64), f.units)
        new_field[sl_center, sl_center, sl_center] = f
        return new_field



yt stores information about the domain of the simulation:

In [12]:
ds.domain_width

yt can also convert this into various units:

In [13]:
print (ds.domain_width.in_units("kpc"))
print (ds.domain_width.in_units("au"))
print (ds.domain_width.in_units("mile"))

[1000.10448889 1000.10448889 1000.10448889] kpc
[2.06286359e+11 2.06286359e+11 2.06286359e+11] au
[1.91755627e+19 1.91755627e+19 1.91755627e+19] mile


Finally, we can get basic information about the particle types and number of particles in a simulation:

In [14]:
print (ds.particle_types)
print (ds.particle_types_raw)
print (ds.particle_type_counts)

['io', 'all']
['io']
{'io': 1124453}


For this dataset, we see that there are two particle types defined, (`io` and `all`), but that only one of these particle types in `ds.particle_types_raw`. The `ds.particle_types` list contains *all* particle types in the simulation, including ones that are dynamically defined like particle unions. The `ds.particle_types_raw` list includes only particle types that are in the output file we loaded the dataset from.

We can also see that there are a bit more than 1.1 million particles in this simulation. Only particle types in `ds.particle_types_raw` will appear in the `ds.particle_type_counts` dictionary.

# Mesh Structure

If you're using a simulation type that has grids (for instance, here we're using an Enzo simulation) you can examine the structure of the mesh.  For the most part, you probably won't have to use this unless you're debugging a simulation or examining in detail what is going on.

##  How are we storing meshes? 

In [15]:
print (ds.index.grid_left_edge)

[[0.         0.         0.        ]
 [0.25       0.21875    0.25      ]
 [0.5        0.21875    0.25      ]
 [0.21875    0.5        0.25      ]
 [0.5        0.5        0.25      ]
 [0.25       0.25       0.5       ]
 [0.5        0.25       0.5       ]
 [0.25       0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.50976562 0.5        0.5       ]
 [0.50976562 0.5        0.50585938]
 [0.50976562 0.53515625 0.50585938]
 [0.52148438 0.53515625 0.50585938]
 [0.52148438 0.54101562 0.52539062]
 [0.52734375 0.53515625 0.50585938]
 [0.50976562 0.5        0.50585938]
 [0.50976562 0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.5       ]
 [0.5        0.5        0.50585938]
 [0.50634766 0.50244141 0.50585938]
 [0.50488281 0.5        0.50585938]
 [0.5        0.50390625 0.50585938]
 [0.5        0.5        0.5 

But, you may have to access information about individual grid objects!  Each grid object mediates accessing data from the disk and has a number of attributes that tell you about it.  The index (`ds.index` here) has an attribute `grids` which is all of the grid objects.

In [16]:
ds.index.grids[1]

EnzoGrid_0002

In [17]:
g = ds.index.grids[1]
print(g)

EnzoGrid_0002


Grids have dimensions, extents, level, and even a list of Child grids.

In [18]:
g.ActiveDimensions

array([16, 18, 16], dtype=int32)

In [19]:
g.LeftEdge, g.RightEdge

(YTArray([0.25   , 0.21875, 0.25   ]) code_length,
 YTArray([0.5, 0.5, 0.5]) code_length)

In [20]:
g.Level

1

In [21]:
g.Children

[EnzoGrid_0145]

## Advanced Grid Inspection

If we want to examine grids only at a given level, we can!  Not only that, but we can load data and take a look at various fields.

*This section can be skipped!*

## Attempting to put an enzo grid into an amr specification 
- not sure if this is the right application for an enzo gird

In [22]:
gs = ds.index.select_grids(ds.index.max_level)

In [23]:
g2 = gs[0]
print (g2)
print (g2.Parent)
print (g2.get_global_startindex())

EnzoGrid_0028
EnzoGrid_0023
[4096 4096 4096]


In [24]:
g2["density"][:,:,0]

YTArray([[1.0136369e-25, 3.4564638e-25, 6.4192590e-25, ...,
          5.9669651e-25, 5.1001470e-25, 4.2170473e-25],
         [1.7664165e-25, 4.3917114e-25, 7.1012795e-25, ...,
          4.8336836e-25, 2.3386585e-25, 1.5441057e-25],
         [2.0398285e-25, 5.0713038e-25, 8.9340174e-25, ...,
          1.5818960e-25, 6.5505691e-26, 4.2683066e-26],
         ...,
         [4.7057491e-25, 2.8565834e-25, 1.6021799e-25, ...,
          9.8956538e-27, 1.0460956e-26, 1.1069824e-26],
         [2.1737538e-25, 1.2614471e-25, 8.5885592e-26, ...,
          9.5149775e-27, 9.9683854e-27, 1.0507010e-26],
         [9.0090510e-26, 6.8866435e-26, 6.3422531e-26, ...,
          9.1754346e-27, 9.5285530e-27, 1.0011084e-26]], dtype=float32) g/cm**3

In [25]:
print ((g2.Parent.child_mask == 0).sum() * 8)
print (g2.ActiveDimensions.prod())

33592
33592


In [26]:
for f in ds.field_list:
    fv = g[f]
    if fv.size == 0: continue
    print (f, fv.min(), fv.max())

('all', 'creation_time') 0.0 s 0.0 s
('all', 'dynamical_time') 0.0 s 0.0 s
('all', 'metallicity_fraction') 0.0 code_metallicity 0.0 code_metallicity
('all', 'particle_index') 495510.0 dimensionless 999626.0 dimensionless
('all', 'particle_mass') 4.2555414545269184e+38 g 4.2555414545269184e+38 g
('all', 'particle_position_x') 0.41541787345992065 code_length 0.4978604680250108 code_length
('all', 'particle_position_y') 0.262784210985574 code_length 0.4940284595802798 code_length
('all', 'particle_position_z') 0.262642536622731 code_length 0.4951038390260019 code_length
('all', 'particle_type') 1.0 dimensionless 1.0 dimensionless
('all', 'particle_velocity_x') -3771602.328267241 cm/s 6406800.047442201 cm/s
('all', 'particle_velocity_y') -9214187.767832078 cm/s 1247333.940700183 cm/s
('all', 'particle_velocity_z') -9018941.986469014 cm/s -186059.66175555275 cm/s
('enzo', 'Average_creation_time') 0.0 dimensionless 0.0 dimensionless
('enzo', 'Bx') 0.0 code_magnetic 0.0 code_magnetic
('enzo',

In [43]:
type(g2)

yt.frontends.enzo.data_structures.EnzoGrid

In [69]:
# the arm specification wants a tuple as input, and not our defined types such as UnitfulArray

enzo_schema = analysis_schema.AMRGridDataSpecification(left_edge= tuple(g2.LeftEdge), 
                                                       right_edge= tuple(g2.RightEdge),
                                                       dimensions= tuple([1.0, 1.0, 1.0]), 
                                                      level = g2.Level, field_data=g2)

In [70]:
enzo_schema

<AMRGridDataSpecification left_edge=(0.5, 0.5, 0.5) right_edge=(0.504638671875, 0.504150390625, 0.503173828125) dimensions=(1, 1, 1) level=8 field_data={('gas', 'density'): YTArray([[[1.01363690e-25, 4.92932516e-24, 6.12061044e-24,…>

In [71]:
enzo_schema.json()

TypeError: keys must be str, int, float, bool or None, not tuple

# Examining Data in Regions

yt provides data object selectors.  In subsequent notebooks we'll examine these in more detail, but we can select a sphere of data and perform a number of operations on it.  yt makes it easy to operate on fluid fields in an object in *bulk*, but you can also examine individual field values.

This creates a sphere selector positioned at the most dense point in the simulation that has a radius of 10 kpc.

## Recreated what we did in our call 

In [54]:
sp = ds.sphere("max", (10, 'kpc'))

yt : [INFO     ] 2020-05-29 14:32:13,523 Max Value is 7.73427e-24 at 0.5040893554687501 0.4998168945312501 0.5001831054687500


In [55]:
sp

YTSphere (galaxy0030): , center=[1.55561975e+24 1.54243494e+24 1.54356506e+24] cm, radius=3.0856775809623245e+22 cm

We can calculate a bunch of bulk quantities.  Here's that list, but there's a list in the docs, too!

In [56]:
list(sp.quantities.keys())

['WeightedAverageQuantity',
 'TotalQuantity',
 'TotalMass',
 'CenterOfMass',
 'BulkVelocity',
 'WeightedVariance',
 'AngularMomentumVector',
 'Extrema',
 'SampleAtMaxFieldValues',
 'MaxLocation',
 'SampleAtMinFieldValues',
 'MinLocation',
 'SpinParameter']

Let's look at the total mass.  This is how you call a given quantity.  yt calls these "Derived Quantities".  We'll talk about a few in a later notebook.

In [57]:
sp.quantities.total_mass()

YTArray([3.27209035e+42, 8.58102338e+43]) g

In [61]:
schema_sphere = analysis_schema.Sphere(center= analysis_schema.UnitfulCoordinate(unit="unitary",
                                                                                 values =[0.5, 0.5, 0.5]), 
                                       radius=analysis_schema.UnitfulValue(unit="kpc", value=50))

In [62]:
schema_sphere

<Sphere center=<UnitfulCoordinate values=[0.5, 0.5, 0.5] unit='unitary'> radius=<UnitfulValue value=50.0 unit='kpc'>>

In [65]:
schema_sphere.json()

'{"center": {"values": [0.5, 0.5, 0.5], "unit": "unitary"}, "radius": {"value": 50.0, "unit": "kpc"}}'

In [66]:
schema_sphere.schema_json()

'{"title": "Sphere", "type": "object", "properties": {"center": {"$ref": "#/definitions/UnitfulCoordinate"}, "radius": {"$ref": "#/definitions/UnitfulValue"}}, "required": ["center", "radius"], "definitions": {"UnitfulCoordinate": {"title": "UnitfulCoordinate", "type": "object", "properties": {"values": {"title": "Values", "default": [0.5, 0.5, 0.5], "minItems": 3, "maxItems": 3, "type": "array", "items": {"type": "number"}}, "unit": {"title": "Unit", "default": "unitary", "type": "string"}}}, "UnitfulValue": {"title": "UnitfulValue", "type": "object", "properties": {"value": {"title": "Value", "type": "number"}, "unit": {"title": "Unit", "default": "unitary", "type": "string"}}, "required": ["value"]}}}'